## 패키지 import

In [ ]:
# 한글 폰트 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (5,586 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 123594 files and dire

In [ ]:
# 폰트 설치 확인
import  matplotlib
import  matplotlib.font_manager  as fm
import  matplotlib.pyplot  as plt


sys_font  = fm.findSystemFonts ( )

[ font  for  font  in  sys_font  if  "Nanum"  in font ]

['/usr/share/fonts/truetype/nanum/NanumSquareRoundB.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicCodingBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf',
 '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareRoundR.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothic.ttf',
 '/usr/share/fonts/truetype/nanum/NanumMyeongjo.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareR.ttf',
 '/usr/share/fonts/truetype/nanum/NanumMyeongjoBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareB.ttf',
 '/usr/share/fonts/truetype/nanum/NanumBarunGothicBold.ttf']

In [ ]:
# 나눔 폰트 설정
font_path = "/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf"

font_name  = fm.FontProperties(fname=font_path, size=12).get_name( )

print("◎ 폰트 이름 : ",font_name)

plt.rc("font", family= font_name)

◎ 폰트 이름 :  NanumGothic


In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install konlpy --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 17.6 MB/s eta 0:00:00


In [ ]:
!pip install nltk --q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
from wordcloud import WordCloud
import re

import tensorflow as tf
import torch
from torch.optim import AdamW, lr_scheduler

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
import nltk
from konlpy.tag import Komoran
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, BertConfig

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


# bert

In [ ]:
# data load

file_path = '/content/drive/MyDrive/유런 24 여름 방학 프로젝트/Test/test_preprocessing_final.csv'
final_df = pd.read_csv(file_path)

last_df = pd.DataFrame()
last_df['title'] = final_df['processed_title']
last_df['label'] = final_df['label']

last_df

,title,label
0,국제 교류 팀 계약직원 모집,1
1,강사 추가 채용,0
2,현대해상 마음 한글 모집,1
3,국제 교류 팀 파견 국제 교류 프로그램 선발 안내,1
4,국민대학교 창업 동아리 모집 안내,1
...,...,...
908,서울대학교 사회봉사 후보 추천 공고,1
909,서울대학교 발전 공로상 후보 추천 공고,1
910,융합과학 기술대학 신입생 후기 모집,0
911,전기 학위 수여식 입학식 안내,0


In [ ]:
y = last_df['label']

In [ ]:
test = ['[CLS]' + str(text) + '[SEP]' for text in last_df['title']]

In [ ]:
# tokenizing using multilingual

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case = False)
tokenized_test = [tokenizer.tokenize(fixed) for fixed in test]

print(tokenized_test[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

['[CLS]', '국제', '교', '##류', '팀', '계', '##약', '##직', '##원', '모', '##집', '[SEP]']


In [ ]:
test_ids = [tokenizer.convert_tokens_to_ids(fixed) for fixed in tokenized_test]

print(test_ids[0]) # 토크나이징 된 단어를 id로 나타냄

[101, 61320, 8907, 46520, 9899, 8887, 47289, 33077, 14279, 9283, 38696, 102]


In [ ]:
# 최대 길이가 128로 설정, 나머지 빈 부분은 0으로 패딩해줌
max_len = 128

# pad_sequences: 시퀀스 데이터를 일정한 길이(max_len)로 맞추는 데 사용
test_ids = pad_sequences(test_ids, maxlen = max_len, dtype = 'long', padding = 'post', truncating = 'post')

print(test_ids[0])

[  101 61320  8907 46520  9899  8887 47289 33077 14279  9283 38696   102
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


In [ ]:
# masking
mask = []
attention_mask = []

for id in test_ids: # test_ids는 이미 토큰화 + 임베딩 + 패딩까지 완료된 상태, id는 한 문장에 들어있는 토큰들의 id list
  mask = [float(i>0) for i in id] # 패딩 토큰(0)에는 0.0, 실제 토큰에는 1.0으로 채워진 마스크 생성함으로써 패딩 토큰은 고려하지 않게 됨
  attention_mask.append(mask) # 각 id list마다 mask가 만들어짐

In [ ]:
# 텐서로 바꿔줌

test_ids = torch.tensor(test_ids)
attention_mask = torch.tensor(attention_mask)
y = torch.tensor(y)

In [ ]:
BATCH_SIZE = 32
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# test setting

test_data = TensorDataset(test_ids, attention_mask, y) # 같은 위치에 있는 데이터를 튜플 형태로 묶어줌
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, batch_size = BATCH_SIZE, sampler = test_sampler)

model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels = 2).to(DEVICE)
model_path = '/content/drive/MyDrive/유런 24 여름 방학 프로젝트/BERT/multilingual/m-bert_ES.pth'
model.load_state_dict(torch.load(model_path))
model.to(DEVICE)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [ ]:
# test

test_loss = 0
test_acc = []
model.eval()

criterion = torch.nn.CrossEntropyLoss()

for batch in test_dataloader:

  batch = tuple(t.to(DEVICE) for t in batch)
  input_ids, input_mask, labels = batch

  with torch.no_grad(): # 기울기 계산 비활성화
    # forward pass
    outputs = model(input_ids, token_type_ids = None, attention_mask = input_mask)
    loss = criterion(outputs.logits, labels)
    test_loss += loss.item()

  pred = np.array([torch.argmax(logit).cpu().detach().item() for logit in outputs.logits])
  true = np.array([label for label in labels.cpu().numpy()])
  accuracy = np.sum(pred == true) / true.shape[0]
  test_acc.append(accuracy)

avg_test_loss = test_loss / len(test_dataloader)
avg_test_acc = np.mean(test_acc)

print("-----test-----")
print(f"test loss: {avg_test_loss:.5f}")
print(f"test acc: {avg_test_acc:.5f}")

-----test-----
test loss: 0.33688
test acc: 0.90530
